In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle

import numpy as np

import xml.etree.ElementTree as ElementTree

`triangle_area` evaluates the triangle area $A$ using Heron's formula:

$$ A = \sqrt{s (s-a) (s-b) (s-c)} \ , $$
where $s$ is the semi-perimeter, $a$ is the length of side a, $b$ is the length of side b, and $c$ is the length of side c.

In [ ]:
def triangle_area(vertices):
    """
    Returns the triangle's area.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle area.
    :return type: float.
    """
    a = np.linalg.norm(vertices[1] - vertices[0])
    b = np.linalg.norm(vertices[2] - vertices[1])
    c = np.linalg.norm(vertices[0] - vertices[2])
    semiperimeter = (a + b + c) * 0.5
    area = np.sqrt(semiperimeter * (semiperimeter - a) * (semiperimeter - b) * (semiperimeter - c))
    return area

`triangle_centroid` evaluates the centroid $C$ of the triangular face following

$$C = \frac{v_a + v_b + v_c}{3} \ , $$
where $v_a$, $v_b$, and $v_c$ are the triangle's vertices.

In [ ]:
def triangle_centroid(vertices):
    """
    Returns the triangle's centroid.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle's centroid.
    :return type: one-dimensional array with 3 columns.
    """
    centroid = (vertices[0] + vertices[1] + vertices[2]) / 3
    return centroid

`triangle_normal` evaluates the face normal of a triangle of vertices $v_a$, $v_b$, and $v_c$ following

$$n = (v_b - v_a) \times (v_c - v_a) \ , $$
where $\times$ represents a cross product.

In [ ]:
def triangle_normal(vertices):
    """
    Returns the face normal of a triangle.

    :param vertices: the vertices of the triangle.
    :type vertices: two-dimensional array.
    :return: the triangle's normal.
    :return type: one-dimensional array with 3 columns.
    """
    side_x = vertices[1] - vertices[0]
    side_y = vertices[2] - vertices[0]
    normal = np.cross(side_x, side_y)
    return normal

`check_visibility` verifies if a plane still allow the two faces to see each other.

In [ ]:
def check_visibility(centroid_a, centroid_b, vertices_c):
    """
    Returns true if the plane still allow the two faces to see each other.

    :param centroid_a: the centroid of face a.
    :type centroid_a: numpy array.
    :param centroid_b: the centroid of face b.
    :type centroid_b: numpy array.
    :param vertices_c: the vertices of the triangle that may be between face a and face b.
    :type vertices_c: numpy array.
    :return: true if the two faces can see each other.
    :return type: bool.
    """
    b = centroid_b - vertices_c[2]
    a = np.array([vertices_c[1] - vertices_c[2], vertices_c[0] - vertices_c[2], centroid_b - centroid_a]).transpose()
    
    if np.linalg.det(a) == 0:
        return True
    
    x = np.linalg.solve(a, b)

    inside_centroid_line = False
    if x[2] >= 0 and x[2] <= 1:
        inside_centroid_line = True

    inside_triangle = False
    if x[0] + x[1] <= 1 and x[0] >= 0 and x[1] >= 0:
        inside_triangle = True

    if inside_triangle and inside_centroid_line:
        return False

    return True

Testing the previous functions.

In [ ]:
vertices_test = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
vertices_test_area = triangle_area(vertices_test)
vertices_test_centroid = triangle_centroid(vertices_test)
vertices_test_normal = triangle_normal(vertices_test)
assert(vertices_test_area - 0.5 < 1e-9)
assert(np.linalg.norm(vertices_test_centroid - np.array([1 / 3, 1 / 3, 0])) < 1e-9)
assert(np.linalg.norm(vertices_test_normal - np.array([0, 0, 1])) < 1e-9)

In [ ]:
centroid_a = np.array([2 / 3, 2 / 3, 0])
centroid_b = np.array([2 / 3, 2 / 3, 1])
vertices_c1 = np.array([[1, 0, 1], [1, 1, 0], [0, 1, 1]])
vertices_c2 = np.array([[1 / 3, 2 / 3, 1], [0, 0, 1], [0, 1, 1]])
assert(check_visibility(centroid_a, centroid_b, vertices_c1) == False)
assert(check_visibility(centroid_a, centroid_b, vertices_c2) == True)

Classes definition.

In [ ]:
class Solid(object):
    """
    Represents a 3D solid.
    """

    def __init__(self, faces, id):
        """
        Creates a new 3 dimensional solid.        
        
        :param faces: the triangular faces of the solid.
        :type faces: TriangularFace.
        :param id: the solid's id.
        :type id: string.
        """
        self.faces = faces
        self.id = id

In [ ]:
class TriangularFace(object):
    """
    Represents a triangular face.
    """

    def __init__(self, vertices, colors, light_source=0):
        """
        Creates a new triangular face.

        :param vertices: the triangle's vertices.
        :type vertices: numpy array.
        :param colors: the color of each vertice.
        :type colors: numpy array.
        :param light_source: if the face is a light source.
        :type light_source: int {0, 1}.
        """
        self.vertices = vertices
        self.colors = colors
        self.rho = np.sum(self.colors, axis=0) / len(self.colors)
        self.area = triangle_area(self.vertices)
        self.centroid = triangle_centroid(self.vertices)
        self.normal = triangle_normal(self.vertices)
        self.light_source = light_source

`form_factor` evaluates the form factor of faces $i$ and $j$ following

$$F_{i, j} = \frac{A_j \cos{\theta_i} \cos{\theta_j}}{\pi r^{2}} \ , $$
where $r$ is the distance between the faces centroid, $A_j$ is the area of face $j$, $\theta_i$ is the angle between the centroid line and the face $i$ normal, and $\theta_j$ is the angle between the centroid line and the face $j$ normal. 

In [ ]:
def form_factor(face_i, face_j, scene_faces):
    """
    Evaluates the form factor of faces i and j, Fij.

    :param face_i: the face i of the scene.
    :type face_i: TriangularFace.
    :param face_j: the face j of the scene.
    :type face_j: TriangularFace.
    :param scene_faces: all the faces in the scene.
    :type scene_faces: list of TriangularFace.
    """
    # Since we have convex solid, then Fii is always zero
    if face_i == face_j:
        return 0

    if scene_faces is not None:
        for face in scene_faces:
            if face == face_i or face == face_j:
                continue

            is_visible = check_visibility(face_i.centroid, face_j.centroid, face.vertices)
            if not is_visible:
                form_factor = 0
                return form_factor

    centroid_line = face_i.centroid - face_j.centroid
    centroid_line_unit = centroid_line / np.linalg.norm(centroid_line)
    normal_i_unit = face_i.normal / np.linalg.norm(face_i.normal)
    normal_j_unit = face_j.normal / np.linalg.norm(face_j.normal)    
    theta_i = np.arccos(np.clip(np.dot(normal_i_unit, -centroid_line_unit), -1.0, 1.0))
    theta_j = np.arccos(np.clip(np.dot(normal_j_unit, centroid_line_unit), -1.0, 1.0))

    # The faces cannot see each other
    if theta_i > np.pi * 0.5 or theta_j > np.pi * 0.5:
        form_factor = 0
        return form_factor

    r = np.linalg.norm(centroid_line)
    area_j = face_j.area
    form_factor = (area_j * np.cos(theta_i) * np.cos(theta_j)) / (np.pi * r**2)
    return form_factor

Testing the previous function.

In [ ]:
vertices_i = np.array([[1, 0, 1], [1, 1, 0], [0, 0, 0]])
vertices_j = np.array([[0, 0, 1], [0, 1, 0], [0, 1, 1]])
colors = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
face_i = TriangularFace(vertices_i, colors)
face_j = TriangularFace(vertices_j, colors)
assert(form_factor(face_i, face_j, None) - 0.1837 < 1e-4)

In [ ]:
vertices_i = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
vertices_j = np.array([[0, 0, 1], [1, 0, 1], [0, 1, 1]])
colors = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
face_i = TriangularFace(vertices_i, colors)
face_j = TriangularFace(vertices_j, colors)
assert(form_factor(face_i, face_j, None) == 0)

In [ ]:
vertices_i = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
vertices_j = np.array([[1, 0, 1], [0, 0, 1], [0, 1, 1]])
colors = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]])
face_i = TriangularFace(vertices_i, colors)
face_j = TriangularFace(vertices_j, colors)
assert(form_factor(face_i, face_j, None) - (1 / (2*np.pi)) < 1e-4)

Parsing the COLLADA file with standard Python library to parse xml.

In [ ]:
scene_path = "/content/drive/My Drive/radiosity/scene.dae"
tree = ElementTree.parse(scene_path)
root = tree.getroot()

Finding the library geometries and the library visual scene sections.

In [ ]:
index = 0
for child in root:
    if "library_geometries" in child.tag:
        library_geometries_index = index

    if "library_visual_scenes" in child.tag:
        library_visual_scenes_index = index

    index += 1
library_geometries = root[library_geometries_index]
library_visual_scenes = root[library_visual_scenes_index]

Get transform matrix based on data of Library Visual Scene.

In [ ]:
transform_matrix = {}
for child in library_visual_scenes[0]:
    for scene_child in child:
        if "sid" in list(scene_child.attrib.keys()):
            matrix = np.array(scene_child.text.split(" ")).reshape((4, 4)).astype(np.float64)
        
        if "url" in list(scene_child.attrib.keys()):
            mesh_name = scene_child.attrib["url"].replace("#", "")
            transform_matrix[mesh_name] = matrix

Create Solid objects based on COLLADA data.

In [ ]:
solid_scene = []
for child in library_geometries:
    id = child.attrib["id"]

    if id == "LightObject-mesh":
        light_source = True
    else:
        light_source = False

    mesh_positions = None
    mesh_colors = None
    mesh_material = None
    for child_geometry in child[0]:
        if "id" in list(child_geometry.attrib.keys()):
            if "mesh-positions" in child_geometry.attrib["id"]:
                mesh_positions = child_geometry
            elif "mesh-colors-Color" in child_geometry.attrib["id"]:
                mesh_colors = child_geometry
        elif "material" in list(child_geometry.attrib.keys()):
            mesh_material = child_geometry

    vertices_array = np.empty((0, 3), np.float64)
    if mesh_positions is not None:
        for child_mesh_positions in mesh_positions:
            if "id" in list(child_mesh_positions.attrib.keys()):
                if "mesh-positions-array" in child_mesh_positions.attrib["id"]:
                    values = child_mesh_positions.text.split(" ")
                    number_of_values = len(values)
                    number_of_vertices = number_of_values // 3
                    for i in range(number_of_vertices):
                        x_coordinate = float(values[i * 3])
                        y_coordinate = float(values[i * 3 + 1])
                        z_coordinate = float(values[i * 3 + 2])
                        coordinate = np.array([x_coordinate, y_coordinate, z_coordinate, 1])
                        coordinate_world = transform_matrix[id] @ coordinate
                        coordinate_world = coordinate_world[:3]
                        vertices_array = np.append(vertices_array, np.array([[x_coordinate, y_coordinate, z_coordinate]]), axis=0)

    face_colors_array = np.empty((0, 4), np.float64)
    if mesh_colors is not None:
        for child_mesh_colors in mesh_colors:
            if "id" in list(child_mesh_colors.attrib.keys()):
                if "mesh-colors-Color-array" in child_mesh_colors.attrib["id"]:
                    values = child_mesh_colors.text.split(" ")
                    number_of_values = len(values)
                    number_of_faces = number_of_values // 12
                    for i in range(number_of_faces):
                        face_color = np.empty((0, 4), np.float64)
                        for j in range(3):
                            red = float(values[i * 4])
                            green = float(values[i * 4 + 1])
                            blue = float(values[i * 4 + 2])
                            alpha = float(values[i * 4 + 3])
                            face_color = np.append(face_color, np.array([[red, green, blue, alpha]]), axis=0)
                        face_colors_array = np.append(face_colors_array, face_color, axis=0)

    if mesh_material is not None:
        number_of_triangles = int(mesh_material.attrib["count"])
        step = 0
        for child_mesh_material in mesh_material:
            if len(list(child_mesh_material.keys())) == 0:
                faces_definition = child_mesh_material.text.split(" ")
                faces_list = []
                for i in range(number_of_triangles):
                    index_vertice_a = int(faces_definition[i * 3 * step])
                    index_vertice_b = int(faces_definition[i * 3 * step + step])
                    index_vertice_c = int(faces_definition[i * 3 * step + 2 * step])

                    color_a = [0, 0, 0, 1]
                    color_b = [0, 0, 0, 1]
                    color_c = [0, 0, 0, 1]
                    if len(face_colors_array) != 0:
                        index_color_a = int(faces_definition[i * 3 * step + step - 1])
                        index_color_b = int(faces_definition[i * 3 * step + 2 * step - 1])
                        index_color_c = int(faces_definition[i * 3 * step + 3 * step - 1])

                        color_a = face_colors_array[index_color_a]
                        color_b = face_colors_array[index_color_b]
                        color_c = face_colors_array[index_color_c]

                    vertice_a = vertices_array[index_vertice_a]
                    vertice_b = vertices_array[index_vertice_b]
                    vertice_c = vertices_array[index_vertice_c]

                    vertices_face = np.array([vertice_a, vertice_b, vertice_c])
                    colors_face = np.array([[color_a], [color_b], [color_c]])
                    face = TriangularFace(vertices_face, colors_face, light_source)
                    faces_list.append(face)
            step += 1

        solid = Solid(faces_list, id)
        solid_scene.append(solid)

Evaluating the form factors $F_{ij}$.

In [ ]:
# Load objects
with open(f"/content/drive/My Drive/radiosity/form_factor_red.pickle", 'rb') as handle:
  form_factor_red = pickle.load(handle)

with open(f"/content/drive/My Drive/radiosity/form_factor_green.pickle", 'rb') as handle:
  form_factor_green = pickle.load(handle)

with open(f"/content/drive/My Drive/radiosity/form_factor_blue.pickle", 'rb') as handle:
  form_factor_blue = pickle.load(handle)

with open(f"/content/drive/My Drive/radiosity/emission_vector.pickle", 'rb') as handle:
  emission_vector = pickle.load(handle)

In [ ]:
# Define indices
# Last run: initial pos - 1900
#           final pos - 2100
INITIAL_POSITION=1900
END_POSITION=2100

In [ ]:
scene_faces = []
for solid in solid_scene:
    scene_faces = [*scene_faces, *solid.faces]

for i in range(INITIAL_POSITION, len(scene_faces)):
    if i == END_POSITION:
      break

    print(f"{i} / {len(scene_faces) - 1}")
    form_factor_i = np.array([])
    for j in range(len(scene_faces)):
        form_factor_i = np.append(form_factor_i, np.array([form_factor(scene_faces[i], scene_faces[j], scene_faces)]))
    form_factor_red = np.append(form_factor_red, (-scene_faces[i].rho[0][0]) * form_factor_i)
    form_factor_green = np.append(form_factor_green, (-scene_faces[i].rho[0][1]) * form_factor_i)
    form_factor_blue = np.append(form_factor_blue, (-scene_faces[i].rho[0][2]) * form_factor_i)
    emission_vector = np.append(emission_vector, int(scene_faces[i].light_source))

1900 / 2093
1901 / 2093
1902 / 2093
1903 / 2093
1904 / 2093
1905 / 2093
1906 / 2093
1907 / 2093
1908 / 2093
1909 / 2093
1910 / 2093
1911 / 2093
1912 / 2093
1913 / 2093
1914 / 2093
1915 / 2093
1916 / 2093
1917 / 2093
1918 / 2093
1919 / 2093
1920 / 2093
1921 / 2093
1922 / 2093
1923 / 2093
1924 / 2093
1925 / 2093
1926 / 2093
1927 / 2093
1928 / 2093
1929 / 2093
1930 / 2093
1931 / 2093
1932 / 2093
1933 / 2093
1934 / 2093
1935 / 2093
1936 / 2093
1937 / 2093
1938 / 2093
1939 / 2093
1940 / 2093
1941 / 2093
1942 / 2093
1943 / 2093
1944 / 2093
1945 / 2093
1946 / 2093
1947 / 2093
1948 / 2093
1949 / 2093
1950 / 2093
1951 / 2093
1952 / 2093
1953 / 2093
1954 / 2093
1955 / 2093
1956 / 2093
1957 / 2093
1958 / 2093
1959 / 2093
1960 / 2093
1961 / 2093
1962 / 2093
1963 / 2093
1964 / 2093
1965 / 2093
1966 / 2093
1967 / 2093
1968 / 2093
1969 / 2093
1970 / 2093
1971 / 2093
1972 / 2093
1973 / 2093
1974 / 2093
1975 / 2093
1976 / 2093
1977 / 2093
1978 / 2093
1979 / 2093
1980 / 2093
1981 / 2093
1982 / 2093
1983

In [ ]:
form_factor_red = np.identity(len(scene_faces)) - form_factor_red.reshape((len(scene_faces), len(scene_faces)))
form_factor_green = np.identity(len(scene_faces)) - form_factor_green.reshape((len(scene_faces), len(scene_faces)))
form_factor_blue = np.identity(len(scene_faces)) - form_factor_blue.reshape((len(scene_faces), len(scene_faces)))

Solving the radiosity equation

In [ ]:
np.unique(np.linalg.solve(form_factor_red, emission_vector))

array([0., 1.])

In [ ]:
np.unique(np.linalg.solve(form_factor_green, emission_vector))

array([0., 1.])

The solution are only 0 or 1 values. In fact, $\mathbf{B}$ equals to $\mathbf{E}$ because the form factors are so small.

In [ ]:
np.unique(form_factor_blue[30])

array([0.00000000e+00, 9.05211365e-36, 9.18054574e-36, 9.50327817e-36,
       9.68639014e-36, 1.02885511e-35, 1.03082903e-35, 1.07119592e-35,
       1.12591475e-35, 1.14756822e-35, 1.14846802e-35, 1.15746203e-35,
       1.19347090e-35, 1.19347091e-35, 1.20134638e-35, 1.21782785e-35,
       1.26367369e-35, 1.29101419e-35, 1.29194587e-35, 1.30990715e-35,
       1.31353807e-35, 1.32607994e-35, 1.32936116e-35, 1.33930694e-35,
       1.39448313e-35, 1.45545240e-35, 1.49183707e-35, 1.52142322e-35,
       1.61279851e-35, 1.61279852e-35, 1.62074822e-35, 1.65759902e-35,
       1.66118743e-35, 1.69082361e-35, 1.69420169e-35, 1.71038638e-35,
       1.75510584e-35, 1.76085870e-35, 1.79349438e-35, 1.83297580e-35,
       1.85193821e-35, 1.86479840e-35, 1.99750288e-35, 2.02562966e-35,
       2.05770635e-35, 2.06562467e-35, 2.21926423e-35, 2.29513634e-35,
       2.29513635e-35, 2.35932911e-35, 2.38694199e-35, 2.38694259e-35,
       2.38694401e-35, 2.41865296e-35, 2.42611933e-35, 2.43014176e-35,
      